# Search the C-STAC API by bounding box or geojson
Items in the API may be searched by a bounding box or GeoJSON. Items intersecting the bounding box or GeoJSON are returned.

## Import dependencies

In [1]:
from typing import List, Dict, Optional
import urllib.parse
import json
import httpx
from ipyleaflet import Map, VectorTileLayer, basemap_to_tiles
import logging


logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

## Base URL for the API
For API documentation, see https://c-stac-api.c-core.app/docs
C-STAC-API implements the STAC API specification. For further details of the API search specification, see https://api.stacspec.org/v1.0.0-rc.1/item-search/

In [2]:
api_url = "https://c-stac-api.c-core.app"

## List all collection IDs in the catalog

In [3]:
def list_collection_ids(url: str = api_url) -> List[str]:
    """List collection IDs in the catalog."""
    get_url = f"{url}/collections"
    
    with httpx.Client(timeout = 20) as client:
        response = client.get(get_url)
    
    try:
        # Raise exception if status code not 2xx success
        response.raise_for_status()
    except httpx.HTTPError as exception:
        log.exception(exception.response.json())
        raise exception
    
    response_body = response.json()
    collections = response_body["collections"]
    collection_ids = [collection["id"] for collection in collections]
    
    return collection_ids
    

In [4]:
collection_ids = list_collection_ids()
collection_ids

['joplin',
 'floe-edge-linestrings',
 'floe-edge-polygons',
 'ice-drift-feature-tracking',
 'floe-edge-convergence',
 'floe-edge-coherence']

## Collection names
The database names differ from the common names assigned to the products for reasons of development history. Here is a map:

- `joplin`: A test collection that is unrelated to the sea ice products.
- `ice-drift-feature-tracking`: Ice tracking.
- `floe-edge-convergence`: Land-fast ice motion.
- `floe-edge-coherence`: Tidal cracks.
- `floe-edge-polygons`: Floe edge as a polygon.
- `floe-edge-linestrings`: Floe edge as a linestring.

## List items in collection
This example uses the GET method, which may be easier for simple listings, compared to the POST method. A POST example appears later.

In [5]:
def list_items( 
    collection_id: str, 
    limit: int = 2,
    sortby: Optional[str] = "-id",
    url: str = api_url
) -> List[Dict]:
    """List items, filtering by geoaptial bbox."""

    # `sortby` need to be urlencoded because it accepts `+`
    url_parameters = f"collections={collection_id}&limit={limit}&sortby={urllib.parse.quote(sortby)}"
    get_collections_url = f"{url}/search?{url_parameters}"
    
    with httpx.Client(timeout = 20) as client:
        log.info(get_collections_url)
        response = client.get(get_collections_url)
    
    try:
        # Raise exception if status code not 2xx success
        response.raise_for_status()
    except httpx.HTTPError as exception:
        log.exception(exception.response.json())
        raise exception
    
    response_body = response.json()
    
    return response_body

Note we can sort by `id`. For descending sort, specify `-id`.

In [36]:
items = list_items("floe-edge-polygons", sortby="-id")
[item["properties"]["datetime"] for item in items["features"]]

INFO:__main__:https://c-stac-api.c-core.app/search?collections=floe-edge-polygons&limit=2&sortby=-id


['2022-03-24T12:18:28+00:00', '2022-03-22T12:34:51+00:00']

For ascending sort, specify `+id`.

In [7]:
items = list_items("floe-edge-linestrings", sortby="+id")
[item["properties"]["datetime"] for item in items["features"]]

INFO:__main__:https://c-stac-api.c-core.app/search?collections=floe-edge-linestrings&limit=2&sortby=%2Bid


['2019-12-16T12:25:42+00:00', '2019-12-18T12:25:42+00:00']

## Filter by location with bbox
This is an example of search by the GET method, which is a convenient way to search by a bounding box (bbox). A search by POST method for more complex queires is demonstarted later.

For further API details, see https://c-stac-api.c-core.app/docs#/default/Search_search_get .
For the API specification, see https://api.stacspec.org/v1.0.0-rc.1/item-search/#operation/getItemSearch .

The GET method accepts a `bbox` parameter. Here, we specify rough bounding boxes for Resolute, Pond Inlet, and the world. The bbox order is `[lower left longitude, lower left latitude, upper right longitude, upper right latitude]`.

In [8]:
bboxes = {
    "resolute": [-97.1, 74.3, -92.5, 75.0],
    "pond_inlet": [-80.6, 72.2, -75.1, 73.2],
    "world": [-180, -90, 180, 90],
}

A convenience function for requesting a search GET method with a bbox.

In [9]:
def list_items_in_bbox(
    bbox: List[float], 
    collection_id: str, 
    limit: int = 2,
    sortby: Optional[str] = "-id",
    url: str = api_url
) -> List[Dict]:
    """List items, filtering by geoaptial bbox."""
    bbox_parameter = ",".join([str(coordinate) for coordinate in bbox])
    
    # `sortby` need to be urlencoded because it accepts `+`
    url_parameters = f"collections={collection_id}&bbox={bbox_parameter}&limit={limit}&sortby={urllib.parse.quote(sortby)}"
    get_collections_url = f"{url}/search?{url_parameters}"
    
    with httpx.Client(timeout = 20) as client:
        log.info(get_collections_url)
        response = client.get(get_collections_url)

    try:
        # Raise exception if status code not 2xx success
        response.raise_for_status()
    except httpx.HTTPError as exception:
        log.exception(exception.response.json())
        raise exception
    
    response_body = response.json()
    
    return response_body

In [10]:
items = list_items_in_bbox(bboxes["pond_inlet"], "floe-edge-linestrings", sortby="-id")
[item["properties"]["datetime"] for item in items["features"]]

INFO:__main__:https://c-stac-api.c-core.app/search?collections=floe-edge-linestrings&bbox=-80.6,72.2,-75.1,73.2&limit=2&sortby=-id


['2022-03-24T12:18:28+00:00', '2022-03-21T11:53:57+00:00']

In [11]:
items = list_items_in_bbox(bboxes["pond_inlet"], "floe-edge-linestrings", sortby="+id")
[item["properties"]["datetime"] for item in items["features"]]

INFO:__main__:https://c-stac-api.c-core.app/search?collections=floe-edge-linestrings&bbox=-80.6,72.2,-75.1,73.2&limit=2&sortby=%2Bid


['2020-10-18T12:18:38+00:00', '2020-10-19T12:11:15+00:00']

We can also sort by `datetime`. Because `id` includes the value of the datetime, sorting by either probably results in the same results.

In [12]:
items = list_items_in_bbox(bboxes["pond_inlet"], "floe-edge-linestrings", sortby="-datetime")
[item["properties"]["datetime"] for item in items["features"]]

INFO:__main__:https://c-stac-api.c-core.app/search?collections=floe-edge-linestrings&bbox=-80.6,72.2,-75.1,73.2&limit=2&sortby=-datetime


['2022-03-24T12:18:28+00:00', '2022-03-21T11:53:57+00:00']

In [13]:
items = list_items_in_bbox(bboxes["pond_inlet"], "floe-edge-linestrings", sortby="+datetime")
[item["properties"]["datetime"] for item in items["features"]]

INFO:__main__:https://c-stac-api.c-core.app/search?collections=floe-edge-linestrings&bbox=-80.6,72.2,-75.1,73.2&limit=2&sortby=%2Bdatetime


['2020-10-18T12:18:38+00:00', '2020-10-19T12:11:15+00:00']

## Filter by location with geojson
This is an example of search by the POST method. It requires providing a JSON body with the search request, but allows for more complex queries, such as intersection with a GeoJSON. 

For further API details, see https://c-stac-api.c-core.app/docs#/default/Search_search_post .
For the API specification, see https://api.stacspec.org/v1.0.0-rc.1/item-search/#operation/postItemSearch

Here, we specify GeoJSON polygons surrounding Resolute, Pond Inlet, and the world.

In [14]:
geojsons = {
    "resolute": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -97.09716796875,
              74.27612190544454
            ],
            [
              -92.5103759765625,
              74.27612190544454
            ],
            [
              -92.5103759765625,
              75.00636121985819
            ],
            [
              -97.09716796875,
              75.00636121985819
            ],
            [
              -97.09716796875,
              74.27612190544454
            ]
          ]
        ]
    },
    "pond_inlet": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -80.606689453125,
              72.18852591070342
            ],
            [
              -75.1025390625,
              72.18852591070342
            ],
            [
              -75.1025390625,
              73.18543401519665
            ],
            [
              -80.606689453125,
              73.18543401519665
            ],
            [
              -80.606689453125,
              72.18852591070342
            ]
          ]
        ]
    },
    "world": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -180,
              -90
            ],
            [
              180,
              -90
            ],
            [
              180,
              90
            ],
            [
              -180,
              90
            ],
            [
              -180,
              -90
            ]
          ]
        ]
    }
}

A convenience function for requesting a search POST method with an intersecting GeoJSON.

In [15]:
def list_items_in_geojson(
    geojson: Dict, 
    collection_id: str, 
    limit: int = 2,
    sortby: Optional[List[Dict]] = None,
    url: str = api_url
) -> List[Dict]:
    """List items, filtering by geoaptial bbox."""
    post_url = f"{url}/search"
    
    data = {
        "intersects": geojson,
        "collections": [collection_id],
        "limit": limit,
        "sortby": sortby,
    }
    
    with httpx.Client(timeout = 20) as client:
        log.info(post_url)
        log.info(data)
        response = client.post(post_url, data=json.dumps(data))
    
    try:
        # Raise exception if status code not 2xx success
        response.raise_for_status()
    except httpx.HTTPError as exception:
        log.exception(exception.response.json())
        raise exception
    
    response_body = response.json()
    
    return response_body

When searching with the POST method, `sortby` accespts an array of dictionaries specifying the `field` and `direction`. Here is a datetime descending sort example.

In [16]:
sortby = [{
    "field": "datetime",
    "direction": "desc",
}]

items = list_items_in_geojson(geojsons["pond_inlet"], "floe-edge-linestrings", sortby=sortby)
[item["properties"]["datetime"] for item in items["features"]]

INFO:__main__:https://c-stac-api.c-core.app/search
INFO:__main__:{'intersects': {'type': 'Polygon', 'coordinates': [[[-80.606689453125, 72.18852591070342], [-75.1025390625, 72.18852591070342], [-75.1025390625, 73.18543401519665], [-80.606689453125, 73.18543401519665], [-80.606689453125, 72.18852591070342]]]}, 'collections': ['floe-edge-linestrings'], 'limit': 2, 'sortby': [{'field': 'datetime', 'direction': 'desc'}]}


['2022-03-24T12:18:28+00:00', '2022-03-21T11:53:57+00:00']

Here is a datetime ascending sourt example.

In [17]:
sortby = [{
    "field": "datetime",
    "direction": "asc",
}]

items = list_items_in_geojson(geojsons["pond_inlet"], "floe-edge-linestrings", sortby=sortby)
[item["properties"]["datetime"] for item in items["features"]]

INFO:__main__:https://c-stac-api.c-core.app/search
INFO:__main__:{'intersects': {'type': 'Polygon', 'coordinates': [[[-80.606689453125, 72.18852591070342], [-75.1025390625, 72.18852591070342], [-75.1025390625, 73.18543401519665], [-80.606689453125, 73.18543401519665], [-80.606689453125, 72.18852591070342]]]}, 'collections': ['floe-edge-linestrings'], 'limit': 2, 'sortby': [{'field': 'datetime', 'direction': 'asc'}]}


['2020-10-18T12:18:38+00:00', '2020-10-19T12:11:15+00:00']

## Example with vector data
Let's plot the most recent floe edge polygon data near Resolute.

In [18]:
sortby = [{
    "field": "datetime",
    "direction": "desc",
}]

items = list_items_in_geojson(geojsons["resolute"], "floe-edge-polygons", sortby=sortby, limit=1)
item = items["features"][0]
item

INFO:__main__:https://c-stac-api.c-core.app/search
INFO:__main__:{'intersects': {'type': 'Polygon', 'coordinates': [[[-97.09716796875, 74.27612190544454], [-92.5103759765625, 74.27612190544454], [-92.5103759765625, 75.00636121985819], [-97.09716796875, 75.00636121985819], [-97.09716796875, 74.27612190544454]]]}, 'collections': ['floe-edge-polygons'], 'limit': 1, 'sortby': [{'field': 'datetime', 'direction': 'desc'}]}


{'type': 'Feature',
 'geometry': {'coordinates': [[[-80.767476, 74.415217],
    [-80.767476, 78.146234],
    [-98.151094, 78.146234],
    [-98.151094, 74.415217],
    [-80.767476, 74.415217]]],
  'type': 'Polygon'},
 'properties': {'datetime': '2021-12-24T13:07:39+00:00'},
 'id': 'floe-edge-polygons-2021-12-18T13:06:50.273Z--2021-12-24T13:07:39.300Z',
 'bbox': [-98.151094, 74.415217, -80.767476, 78.146234],
 'assets': {'geojson': {'title': 'GeoJSON',
   'description': 'Vector data in GeoJSON format',
   'start_datetime': None,
   'end_datetime': None,
   'created': None,
   'updated': None,
   'platform': None,
   'instruments': None,
   'constellation': None,
   'mission': None,
   'providers': None,
   'gsd': None,
   'href': 'https://coresight-layers.s3-us-west-2.amazonaws.com/floe-edge-polygons/2021-12-18T13:06:50.273Z--2021-12-24T13:07:39.300Z/2021-12-18T13:06:50.273Z--2021-12-24T13:07:39.300Z.geojson',
   'type': 'application/geo+json',
   'roles': ['vector-data']},
  'vector_til

## Convenience functions
A few convenience functions to extract the vector tile url and vector tile metadata from the STAC item.

In [19]:
def get_vector_tiles_url(item: Dict) -> Optional[str]:
    """Get vector tiles url from STAC item"""
    try:
        url = item["assets"]["vector_tiles"]["href"]
    except KeyError:
        log.warning(f"Item {item['id']} has no vector tile asset")
        return None
    
    return url

In [20]:
def get_vector_tiles_metadata(item: Dict) -> Optional[Dict]:
    """Get vector tiles metadata from STAC item"""
    try:
        url = item["assets"]["vector_tiles_metadata"]["href"]
    except KeyError:
        log.warning(f"Item {item['id']} has no vector tile metadata asset")
        return None
    
    with httpx.Client(timeout = 20) as client:
        log.info(url)
        response = client.get(url)
    
    try:
        # Raise exception if status code not 2xx success
        response.raise_for_status()
    except httpx.HTTPError as exception:
        log.exception(exception.response.json())
        raise exception
    
    response_body = response.json()
    
    return response_body

Get the vector tile url and metadata.

In [21]:
url = get_vector_tiles_url(item)
url

'https://coresight-layers.s3-us-west-2.amazonaws.com/floe-edge-polygons/2021-12-18T13:06:50.273Z--2021-12-24T13:07:39.300Z/{z}/{x}/{y}.pbf'

In [22]:
metadata = get_vector_tiles_metadata(item)
metadata

INFO:__main__:https://coresight-layers.s3-us-west-2.amazonaws.com/floe-edge-polygons/2021-12-18T13:06:50.273Z--2021-12-24T13:07:39.300Z/metadata.json


{'name': '/tmp/tmpcmow0252',
 'description': '/tmp/tmpcmow0252',
 'version': '2',
 'minzoom': '1',
 'maxzoom': '10',
 'center': '-81.035156,76.393250,10',
 'bounds': '-98.151094,74.415217,-80.767476,78.146234',
 'type': 'overlay',
 'format': 'pbf',
 'generator': 'tippecanoe v1.36.0',
 'generator_options': "tippecanoe '--minimum-zoom=1' '--maximum-zoom=10' -l floe-edge-polygons --output-to-directory /tmp/tmpcmow0252 --drop-densest-as-needed --extend-zooms-if-still-dropping --no-tile-compression --read-parallel --force /tmp/tmpcmow0252/2021-12-18T13:06:50.273Z--2021-12-24T13:07:39.300Z.geojson",
 'json': '{"vector_layers": [ { "id": "floe-edge-polygons", "description": "", "minzoom": 1, "maxzoom": 10, "fields": {"datetime_interval": "String", "geometry_type": "String", "ogc_fid": "Number", "timedelta_seconds": "Number", "timestamp1": "String", "timestamp2": "String"} } ],"tilestats": {"layerCount": 1,"layers": [{"layer": "floe-edge-polygons","count": 5,"geometry": "Polygon","attributeCou

In [23]:
center = [float(coordinate) for coordinate in metadata["center"].split(",")[:2]][::-1]
center

[76.39325, -81.035156]

## Plot vector data

In [24]:
layer = VectorTileLayer(url=url)
map = Map(center=center, zoom=6)
map.add_layer(layer)

map

Map(center=[76.39325, -81.035156], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

## Example with raster data
Let's plot the most recent tidal crack data near Resolute.

In [25]:
sortby = [{
    "field": "datetime",
    "direction": "desc",
}]

items = list_items_in_geojson(geojsons["resolute"], "floe-edge-coherence", sortby=sortby, limit=1)
item = items["features"][0]
item

INFO:__main__:https://c-stac-api.c-core.app/search
INFO:__main__:{'intersects': {'type': 'Polygon', 'coordinates': [[[-97.09716796875, 74.27612190544454], [-92.5103759765625, 74.27612190544454], [-92.5103759765625, 75.00636121985819], [-97.09716796875, 75.00636121985819], [-97.09716796875, 74.27612190544454]]]}, 'collections': ['floe-edge-coherence'], 'limit': 1, 'sortby': [{'field': 'datetime', 'direction': 'desc'}]}


{'type': 'Feature',
 'geometry': {'coordinates': [[[74.3381679945762, -85.65939696533742],
    [74.3381679945762, 78.06458264850178],
    [-92.62825799363675, 78.06458264850178],
    [-92.62825799363675, -85.65939696533742],
    [74.3381679945762, -85.65939696533742]]],
  'type': 'Polygon'},
 'properties': {'datetime': '2021-09-03T12:50:29+00:00'},
 'id': '20210903T125029_20210915T125030_001664_swath4_fasticecoherence',
 'bbox': [-92.62825799363675,
  -85.65939696533742,
  74.3381679945762,
  78.06458264850178],
 'assets': {'geotiff': {'title': 'GeoTIFF',
   'description': 'Raster data in GeoTIFF format',
   'start_datetime': None,
   'end_datetime': None,
   'created': None,
   'updated': None,
   'platform': None,
   'instruments': None,
   'constellation': None,
   'mission': None,
   'providers': None,
   'gsd': None,
   'href': 'https://s3.ca-central-1.amazonaws.com/c-core-ottawa/20210903T125029_20210915T125030_001664_swath4_fasticecoherence-coh.tif',
   'type': 'image/tiff; appli

## Convenience functions
Convenience function to get the raster tile url from the STAC item. Note, the raster tiles url are in TMS schema, as opposed to XYZ schema, so we need to invert the `y` value (https://gist.github.com/tmcw/4954720).

In [26]:
def get_raster_tiles_url(item: Dict, tms: Optional[bool] = None) -> Optional[str]:
    """Get raster tiles url from STAC item"""
    try:
        url = item["assets"]["raster_tiles"]["href"]
    except KeyError:
        log.warning(f"Item {item['id']} has no raster tile asset")
        return None
    
    # If url is TMS scheme, need to invert the `y`.
    # https://gist.github.com/tmcw/4954720?permalink_comment_id=3768325#gistcomment-3768325
    if tms:
        url = url.replace("{y}", "{-y}")
    
    return url

## Plot raster data

In [27]:
url = get_raster_tiles_url(item, tms=True)

center = (item["geometry"]["coordinates"][0][0][0], item["geometry"]["coordinates"][0][0][1])

layer = basemap_to_tiles({
    "url": url,
})
map = Map(center=center, zoom=6)
map.add_layer(layer)

map

Map(center=[74.3381679945762, -85.65939696533742], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [28]:
url

'https://c-core-ottawa.s3.amazonaws.com/tms/20210903T125029_20210915T125030_001664_swath4_fasticecoherence-coh.tif/{z}/{x}/{-y}.png'

In [35]:
items = list_items("floe-edge-polygons", sortby="+id")
item = items["features"][0]
item

INFO:__main__:https://c-stac-api.c-core.app/search?collections=floe-edge-polygons&limit=2&sortby=%2Bid


{'type': 'Feature',
 'geometry': {'coordinates': [[[-75.99302, 74.45437],
    [-75.99302, 76.9875],
    [-85.955635, 76.9875],
    [-85.955635, 74.45437],
    [-75.99302, 74.45437]]],
  'type': 'Polygon'},
 'properties': {'datetime': '2019-12-16T12:25:42+00:00'},
 'id': 'floe-edge-polygons-2019-12-16T12:25:42.821Z--2019-12-17T12:18:19.068Z',
 'bbox': [-85.955635, 74.45437, -75.99302, 76.9875],
 'assets': {'geojson': {'title': 'GeoJSON',
   'description': 'Vector data in GeoJSON format',
   'start_datetime': None,
   'end_datetime': None,
   'created': None,
   'updated': None,
   'platform': None,
   'instruments': None,
   'constellation': None,
   'mission': None,
   'providers': None,
   'gsd': None,
   'href': 'https://coresight-layers.s3-us-west-2.amazonaws.com/floe-edge-polygons/2019-12-16T12:25:42.821Z--2019-12-17T12:18:19.068Z/2019-12-16T12:25:42.821Z--2019-12-17T12:18:19.068Z.geojson',
   'type': 'application/geo+json',
   'roles': ['vector-data']},
  'vector_tiles': {'title':

In [31]:
def get_geojson(item: Dict) -> Optional[Dict]:
    """Get vector tiles metadata from STAC item"""
    try:
        url = item["assets"]["geojson"]["href"]
    except KeyError:
        log.warning(f"Item {item['id']} has no geojson asset")
        return None
    
    with httpx.Client(timeout = 20) as client:
        log.info(url)
        response = client.get(url)
    
    try:
        # Raise exception if status code not 2xx success
        response.raise_for_status()
    except httpx.HTTPError as exception:
        log.exception(exception.response.json())
        raise exception
    
    response_body = response.json()
    
    return response_body

In [33]:
geojson = get_geojson(item)
# geojson

INFO:__main__:https://coresight-layers.s3-us-west-2.amazonaws.com/floe-edge-polygons/2022-03-12T12:18:28.447Z--2022-03-24T12:18:28.833Z/2022-03-12T12:18:28.447Z--2022-03-24T12:18:28.833Z.geojson
